In [2]:
pip install yt_dlp pydub yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 40.4 MB/s eta 0:00:00


In [3]:
import os
import yt_dlp
import cv2
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from pydub import AudioSegment
from collections import Counter
import logging
logging.getLogger("absl").setLevel(logging.ERROR)

In [4]:
# ฟังก์ชันดาวน์โหลดและแปลงไฟล์ .mp3 จาก YouTube
def download_audio_from_youtube(url, output_dir):
    print('')
    try:
        os.makedirs(output_dir, exist_ok=True)
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': os.path.join(output_dir, 'audio.%(ext)s'),
            'keepvideo': False,
            'quiet': True,
            'no_warnings': True,
            'logtostderr': False,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            output_file = os.path.join(output_dir, 'audio.mp3')
            return output_file, info['title']
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการดาวน์โหลด: {e}")
        return None, None

In [5]:
# ฟังก์ชันตัดไฟล์เสียงเป็นช่วงละ 30 วินาที
def trim_audio(input_file, output_dir, duration=30):
    try:
        os.makedirs(output_dir, exist_ok=True)
        audio = AudioSegment.from_file(input_file)
        total_duration = len(audio) / 1000
        segments = []

        for i in range(0, int(total_duration), duration):
            start_time = i * 1000
            end_time = (i + duration) * 1000
            if end_time - start_time == duration * 1000:
                segment = audio[start_time:end_time]
                output_file = os.path.join(output_dir, f"segment_{i // duration + 1}.mp3")
                segment.export(output_file, format="mp3")
                segments.append(output_file)

        return segments
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการตัดไฟล์เสียง: {e}")
        return []

In [6]:
# ฟังก์ชันสร้าง Mel Spectrogram
def create_mel_spectrogram(input_file, output_file):
    try:
        y, sr = librosa.load(input_file, sr=None)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
        mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mel_spectrogram_db, sr=sr, hop_length=512, x_axis=None, y_axis=None, cmap='magma')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
        plt.close()
        return output_file
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการสร้าง Mel Spectrogram: {e}")
        return None

In [7]:
# ฟังก์ชันโหลดและปรับขนาด Mel Spectrogram
def preprocess_mel_spectrogram(img_path):
    try:
        img = load_img(img_path)
        img_array = img_to_array(img)
        img_array = cv2.resize(img_array, (969, 370))
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการปรับขนาดภาพ: {e}")
        return None

In [8]:
# ฟังก์ชันทำนายอารมณ์ของแต่ละช่วง
def predict_emotion(model, spectrogram_file, class_names):
    try:
        mel_img = preprocess_mel_spectrogram(spectrogram_file)
        if mel_img is not None:
            prediction = model.predict(mel_img)
            predicted_class = class_names[np.argmax(prediction)]
            print(f"อารมณ์ที่คาดการณ์: {predicted_class}\n")
            return predicted_class
        return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการทำนายอารมณ์: {e}")
        return None

In [9]:
# ฟังก์ชันทำนายอารมณ์ทั้งเพลง
def predict_song_emotion(model_path, segments, class_names, output_dir):
    try:
        model = load_model(model_path)
        predictions = []

        for i, segment in enumerate(segments):
            spectrogram_file = os.path.join(output_dir, f"mel_spectrogram_{i + 1}.png")
            create_mel_spectrogram(segment, spectrogram_file)

            print(f"ส่วนที่ {i+1} ({str((i*30)//60).zfill(2)}:{str((i*30)%60).zfill(2)}-{str(((i+1)*30)//60).zfill(2)}:{str(((i+1)*30)%60).zfill(2)})")
            predicted_class = predict_emotion(model, spectrogram_file, class_names)
            if predicted_class:
                predictions.append(predicted_class)

        if predictions:
            most_common_emotion = Counter(predictions).most_common(1)[0][0]
            return most_common_emotion
        else:
            print("ไม่มีผลการทำนาย")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการทำนายอารมณ์ของทั้งเพลง: {e}")
        return None

In [10]:
# ฟังก์ชันสำหรับการประมวลผลใน GUI
def process_song(url, model_path, class_names):
    try:
        output_dir = "./outputs"
        downloaded_audio = download_audio_from_youtube(url, output_dir)
        if downloaded_audio:
            segments = trim_audio(downloaded_audio, output_dir, duration=30)
            result = predict_song_emotion(model_path, segments, class_names, output_dir)
            messagebox.showinfo("ผลลัพธ์", f"อารมณ์โดยรวมของเพลง: {result}")
        else:
            messagebox.showerror("ข้อผิดพลาด", "ไม่สามารถดาวน์โหลดไฟล์เสียงได้")
    except Exception as e:
        messagebox.showerror("ข้อผิดพลาด", f"เกิดข้อผิดพลาด: {str(e)}")

In [11]:
# ฟังก์ชันหลัก
def main():
    youtube_url = input("กรุณาใส่ URL ของเพลง YouTube: ").strip()
    if not youtube_url:
        print("URL ไม่สามารถว่างได้")
        return

    model_path = "Mel_Spectrogram_model.h5"
    class_names = ["Happy", "Sad", "Angry", "Calm"]

    output_dir = "./outputs"
    downloaded_audio, song_title = download_audio_from_youtube(youtube_url, output_dir)
    if downloaded_audio:
        print(f"กำลังวิเคราะห์อารมณ์เพลง: {song_title}\n")
        segments = trim_audio(downloaded_audio, output_dir, duration=30)
        result = predict_song_emotion(model_path, segments, class_names, output_dir)
        if result:
            print(f"อารมณ์โดยรวมของเพลง: {result}")
        else:
            print("ไม่สามารถทำนายอารมณ์ได้")
    else:
        print("ไม่สามารถดาวน์โหลดไฟล์เสียงได้")

In [48]:
if __name__ == "__main__":
    main()

กรุณาใส่ URL ของเพลง YouTube: https://www.youtube.com/watch?v=izLHRA9k1jQ

กำลังวิเคราะห์อารมณ์เพลง: Klear - แด่เธอที่รัก (Official MV)

ส่วนที่ 1 (00:00-00:30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 2 (00:30-01:00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 3 (01:00-01:30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 4 (01:30-02:00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 5 (02:00-02:30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
อารมณ์ที่คาดการณ์: Calm

ส่วนที่ 6 (02:30-03:00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 7 (03:00-03:30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 8 (03:30-04:00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 9 (04:00-04:30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
อารมณ์ที่คาดการณ์: Sad

ส่วนที่ 10 (04:30-05:00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
อารมณ์ที่คาดการณ์: Angry

อารมณ์โดยรวมของเพลง